# Value gradient error for linear policies in LQG

**Objective:** compare ground-truth vs. estimated gradients for linear policies to see if it's at all possible to learn a model that predicts the SVG with no concern for sample and computational efficiency.

**Procedure:** for several LQG dimensions, we collect $X$ trajectories from the sampled policy. These trajectories are then used to train a model by optimizing the model learning objective: 
1. we split the data into training, validation and test sets;
2. we choose a batch size and loss improvement threshold;
3. we train the model by stochastic optimization of the model parameters on the learning objective;
4. we stop learning when the validation loss stops improving (we use the PyTorch Lightning framework for this).

**Extensions:** interpolate between optimal and random policies while logging the SVG error.
